In [1]:
import numpy as np
import cv2
from itertools import cycle
import pathlib
import math
import tqdm
import scipy.io
from matplotlib import pyplot as plt
import scipy.io
import h5py
import re
from lxml import etree as ET
import scipy.signal as sig
import pandas as pd
from scipy.stats import kde
from BlockSync_current import BlockSync
import UtilityFunctions_newOE as uf
from scipy import signal
import bokeh

In [2]:
# new utility functions defined here:
def bokeh_plotter(data_list, label_list,
                  plot_name='default',
                  x_axis='X', y_axis='Y',
                  peaks=None, export_path=False):
    """Generates an interactive Bokeh plot for the given data vector.
    Args:
        data_list (list or array): The data to be plotted.
        label_list (list of str): The labels of the data vectors
        plot_name (str, optional): The title of the plot. Defaults to 'default'.
        x_axis (str, optional): The label for the x-axis. Defaults to 'X'.
        y_axis (str, optional): The label for the y-axis. Defaults to 'Y'.
        peaks (list or array, optional): Indices of peaks to highlight on the plot. Defaults to None.
        export_path (False or str): when set to str, will output the resulting html fig
    """
    color_cycle = cycle(bokeh.palettes.Category10_10)
    fig = bokeh.plotting.figure(title=f'bokeh explorer: {plot_name}',
                                x_axis_label=x_axis,
                                y_axis_label=y_axis,
                                plot_width=1500,
                                plot_height=700)

    for i, vec in enumerate(range(len(data_list))):
        color = next(color_cycle)
        data_vector = data_list[vec]
        if label_list is None:
            fig.line(range(len(data_vector)), data_vector, line_color=color, legend_label=f"Line {len(fig.renderers)}")
        elif len(label_list) == len(data_list):
            fig.line(range(len(data_vector)), data_vector, line_color=color, legend_label=f"{label_list[i]}")

    if peaks is not None:
        fig.circle(peaks, data_vector[peaks], size=10, color='red')

    if export_path is not False:
        print(f'exporting to {export_path}')
        bokeh.io.output.output_file(filename=str(export_path / f'{plot_name}.html'), title=f'{plot_name}')
    bokeh.plotting.show(fig)
    
    
    # verification function for a single eye video + ellipse (requires cv2)

# currently working very slowly
def play_video_with_ellipses(block, eye, path_to_video=False, xflip=False):

    if eye == 'left':
        video_path = block.le_videos[0]
        ellipse_dataframe = block.le_df
    elif eye == 'right':
        video_path = block.re_videos[0]
        ellipse_dataframe = block.re_df
    else:
        raise ValueError(f"eye can only be 'left' or 'right'")
    if video_path is not False:
        video_path = path_to_video
    # Open the video file
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error opening video file.")
        return

    # Loop through each frame
    while True:
        # Read a frame from the video
        ret, frame = cap.read()
    
        if not ret:
            # Break the loop if the video is finished
            break
        
        # Optionally flip the frame along the x-axis
        if xflip:
            frame = cv2.flip(frame, 1)

        # Get the corresponding ellipse data for the current frame
        current_frame_num = int(cap.get(cv2.CAP_PROP_POS_FRAMES)) - 1
        try:
            if eye == 'right':
                current_frame_data = ellipse_dataframe.iloc[ellipse_dataframe.query('R_eye_frame == @current_frame_num').index[0]]
            elif eye == 'left':
                current_frame_data = ellipse_dataframe.iloc[ellipse_dataframe.query('L_eye_frame == @current_frame_num').index[0]]
        except IndexError:
            continue
        # Extract ellipse parameters
        try:
            center_x = int(current_frame_data['center_x'])
            center_y = int(current_frame_data['center_y'])
            width = int(current_frame_data['width'])
            height = int(current_frame_data['height'])
            phi = float(current_frame_data['phi'])
    
            # Draw the ellipse on the frame
            cv2.ellipse(frame, (center_x, center_y), (width, height), phi, 0, 360, (0, 255, 0), 2)
    
            # Display the frame
            cv2.imshow('Video with Ellipses', frame)
        
            # Check for the 'q' key to quit
            if cv2.waitKey(25) & 0xFF == ord('q'):
                break
        except ValueError:
            continue
    # Release video capture object and close the window
    cap.release()
    cv2.destroyAllWindows()
    
def play_video_with_ellipses_rotation(block, eye, path_to_video=False, xflip=False, transformation_matrix=None):
    if eye == 'left':
        video_path = block.le_videos[0]
        ellipse_dataframe = block.le_df
    elif eye == 'right':
        video_path = block.re_videos[0]
        ellipse_dataframe = block.re_df
    else:
        raise ValueError(f"eye can only be 'left' or 'right'")
    
    if video_path is not False:
        video_path = path_to_video

    # Open the video file
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error opening video file.")
        return

    # Loop through each frame
    while True:
        # Read a frame from the video
        ret, frame = cap.read()
    
        if not ret:
            # Break the loop if the video is finished
            break
        
        # Optionally flip the frame along the x-axis
        if xflip:
            frame = cv2.flip(frame, 1)

        # Apply transformation matrix if provided
        if transformation_matrix is not None:
            frame = cv2.warpAffine(frame, transformation_matrix, (frame.shape[1], frame.shape[0]))

        # Get the corresponding ellipse data for the current frame
        current_frame_num = int(cap.get(cv2.CAP_PROP_POS_FRAMES)) - 1
        try:
            if eye == 'right':
                current_frame_data = ellipse_dataframe.iloc[ellipse_dataframe.query('R_eye_frame == @current_frame_num').index[0]]
            elif eye == 'left':
                current_frame_data = ellipse_dataframe.iloc[ellipse_dataframe.query('L_eye_frame == @current_frame_num').index[0]]
        except IndexError:
            continue

        # Extract ellipse parameters
        if transformation_matrix is not None:
            try:
                center_x = int(current_frame_data['center_x_rotated'])
                center_y = int(current_frame_data['center_y_rotated'])
                width = int(current_frame_data['width'])
                height = int(current_frame_data['height'])
                phi = float(current_frame_data['phi_rotated'])
                
                # Draw the ellipse on the frame
                cv2.ellipse(frame, (center_x, center_y), (width, height), phi, 0, 360, (0, 255, 0), 2)
                
                # Add text to the frame
                text = f'ellipse angle: {phi}'
                cv2.putText(frame, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        
                
                # Display the frame
                cv2.imshow('Video with Ellipses', frame)
            
                # Check for the 'q' key to quit
                if cv2.waitKey(25) & 0xFF == ord('q'):
                    break
            except ValueError:
                continue
        else:
            try:
                center_x = int(current_frame_data['center_x'])
                center_y = int(current_frame_data['center_y'])
                width = int(current_frame_data['width'])
                height = int(current_frame_data['height'])
                phi = float(current_frame_data['phi'])
        
                # Draw the ellipse on the frame
                cv2.ellipse(frame, (center_x, center_y), (width, height), phi, 0, 360, (0, 255, 0), 2)
                
                # Add text to the frame
                text = f'ellipse angle: {phi}'
                cv2.putText(frame, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                
                # Display the frame
                cv2.imshow('Video with Ellipses', frame)
            
                # Check for the 'q' key to quit
                if cv2.waitKey(25) & 0xFF == ord('q'):
                    break
            except ValueError:
                continue

    # Release video capture object and close the window
    cap.release()
    cv2.destroyAllWindows()

def collect_lights_out_events(data, roll_w_size=1500, plot=False): 
    """Identifies potential lights-out events from the given data.
    
    Args:
        data (list or array): The data containing light measurements.
        roll_w_size (int, optional): The window size for rolling z-score calculation. Defaults to 1500.

    Returns:
        list: Indices of the identified potential lights-out events.
    """
    # use a function to get relative z-scores and deal with changes in ambient light
    z_score_data = rolling_window_z_scores(data, roll_w_size=roll_w_size)
    z_score_data = z_score_data[:len(data)]
    
    # detect peaks based on the scipy algorithm
    peak_indices, _ = scipy.signal.find_peaks(-1 * z_score_data, width=1, distance = 3000)
    
    # expand the peaks to include the dimming and re-lighting frames
    expanded_indices = np.sort(np.array([peak_indices -1, peak_indices, peak_indices + 1]).flatten())
    
    if plot:
        bokeh_plotter([z_score_data],label_list=['phi diff'], plot_name='peak detector output', x_axis='Frame', y_axis='brightness Z score', peaks=expanded_indices)
    
    return expanded_indices

def rolling_window_z_scores(data, roll_w_size=120):
    """
    Detect threshold-crossing data points in a 1D data vector using a rolling window approach.

    Parameters:
    - data (numpy array): 1D data vector with values.
    - roll_w_size (int): Size, in samples, of the rolling window.
    
    Returns:
    - numpy array: A 1D array where each element is the relative z-score of the original value in its window
    """
    result = []
    len_data = len(data)

    for i in tqdm.tqdm(range(0, len_data - roll_w_size + 1, roll_w_size)):
        window_data = data[i:i + roll_w_size]
        
        std_value = np.std(window_data)
        zscores = scipy.stats.zscore(window_data)
        
        
        #threshold_crossing_indices = np.where(window_data < std_value*threshold)[0]
        if i==0:
            result = zscores
        else:
            result = np.concatenate([result, zscores])
        

    # Handle remaining elements after the last complete rolling window
    last_window_start = len_data - roll_w_size
    last_window_data = data[last_window_start:]

    std_value_last = np.std(last_window_data)
    zscores_last = scipy.stats.zscore(last_window_data)
    result = np.concatenate([result, zscores_last])
    
    return result


In [3]:
# define a single block to figure things out with:
# this step creates block_collection - a list of BlockSync objects of interest
block_numbers = range(24,25)
bad_blocks = [42, 61, 62, 64, 65, 66] # True for PV_62
experiment_path = pathlib.Path(r"Z:\Nimrod\experiments")
animal = 'PV_62'
block_collection = uf.block_generator(block_numbers=block_numbers,
                                      experiment_path=experiment_path,
                                      animal=animal,
                                      bad_blocks=bad_blocks)
# create a block_dict object for ease of access:
block_dict = {}
for b in block_collection:
    block_dict[str(b.block_num)] = b



instantiated block number 024 at Path: Z:\Nimrod\experiments\PV_62\2023_04_27\block_024, new OE version
Found the sample rate for block 024 in the xml file, it is 20000 Hz
created the .oe_rec attribute as an open ephys recording obj with get_data functionality
retrieving zertoh sample number for block 024
got it!


In [4]:
# This step is used to quickly go over the analyzed blocks and load their internal data
for block in block_collection:
    block.parse_open_ephys_events()
    block.get_eye_brightness_vectors()
    block.synchronize_block()
    block.create_eye_brightness_df(threshold_value=20)

running parse_open_ephys_events...
block 024 has a parsed events file, reading...
getting eye brigtness values for block 024...
found a file!
blocksync_df loaded from analysis folder
eye_brightness_df loaded from analysis folder


In [5]:
# This is a continuation of the previous - more data loadings
for block in block_collection:
    block.import_manual_sync_df()
    block.read_dlc_data()

eye dataframes loaded from analysis folder


# I want to create a datastream_map object that will hold information about different data sources and their mapping onto the block timeline

In [25]:
sync_map = block.final_sync_df
sync_map = sync_map[['Arena_TTL', 'Arena_frame', 'L_eye_frame','R_eye_frame']]
sync_map = sync_map.rename(columns={'Arena_TTL':'OE_timestamp'})
df = block.le_df[[]
demo_stream = DataStream(df=df,own_sync_map=sync_map[['OE_timestamp', 'L_eye_frame']] )



In [34]:
block.le_df


,Arena_TTL,Unnamed: 0.1,L_eye_frame,L_values,R_values,center_x,center_y,width,height,phi,ellipse_size,ms_axis,center_x_corrected,center_y_corrected
0,605092.0,0,NaN,NaN,0.885172,NaN,NaN,NaN,NaN,NaN,NaN,30254.60,NaN,NaN
1,605431.0,1,NaN,NaN,0.887300,NaN,NaN,NaN,NaN,NaN,NaN,30271.55,NaN,NaN
2,605771.0,2,NaN,NaN,0.890668,NaN,NaN,NaN,NaN,NaN,NaN,30288.55,NaN,NaN
3,606111.0,3,NaN,NaN,0.892202,NaN,NaN,NaN,NaN,NaN,NaN,30305.55,NaN,NaN
4,606451.0,4,3.0,-2.813021,0.893261,315.466003,176.975903,46.355792,37.094571,-0.509178,5402.120118,30322.55,315.466003,176.975903
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91948,31841220.0,91948,93878.0,-1.851355,1.670915,409.768703,142.412726,44.392050,28.938988,0.320629,4035.881617,1592061.00,405.768703,142.412726
91949,31841560.0,91949,93879.0,-1.855220,1.670836,409.593340,142.371146,44.201880,28.903984,0.317515,4013.731602,1592078.00,405.593340,142.371146
91950,31841900.0,91950,93880.0,-1.851100,1.670840,409.575137,142.392658,44.242473,28.899884,0.316178,4016.847748,1592095.00,405.575137,142.392658
91951,31842239.0,91951,93881.0,-1.852275,1.670417,409.573627,142.381196,44.153392,28.857153,0.317981,4002.832648,1592111.95,405.573627,142.381196


# I want to understand how to translate my data such that the X and Y axes are reliable and not determined by the camera tilt
This will be achieved by the following solution:
1. get a line that cuts the image along the tearducts of the animal when the pupil is as close to a circle as possible
2. translate phi, x-center and y-center together with the line such that the line is horizontal

In [7]:
# this is the second iteration of the rotation verification function:
def rotate_frame_to_horizontal_with_interpolation(path_to_video_file, frame_number, ellipse_df, xflip=True, output_path= None):
    """
    Rotate the specified frame from a video file to horizontal orientation with interpolation.

    Parameters:
    - path_to_video_file (str): Path to the video file.
    - frame_number (int): Frame number to be processed.
    - ellipse_df (pd.DataFrame): DataFrame containing ellipse parameters for each frame.
    - xflip (bool, optional): Flag to horizontally flip the frame (default is True).
    - output_path (str, optional): Path to save the output video file (default is None).

    Returns:
    - rotation_matrix (np.ndarray): The rotation matrix used for the transformation.
    - angle (float): The rotation angle applied to the frame.
    """
    # Read the video file
    cap = cv2.VideoCapture(path_to_video_file)

    # Check if the video file is opened successfully
    if not cap.isOpened():
        print("Error: Unable to open video file.")
        return None

    # Set the frame position
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)

    # Read the frame
    ret, frame = cap.read()

    # Check if the frame is read successfully
    if not ret:
        print(f"Error: Unable to read frame {frame_number}.")
        cap.release()
        return None

    # horizontally flip frame if applicable:
    if xflip:
        frame = cv2.flip(frame, 1)

    # get the original ellipse from the block dataframe
    if 'R_eye_frame' in ellipse_df.columns:
        current_frame_data = ellipse_df.iloc[ellipse_df.query('R_eye_frame == @frame_number').index[0]]
    elif 'L_eye_frame' in ellipse_df.columns:
        current_frame_data = ellipse_df.iloc[ellipse_df.query('L_eye_frame == @frame_number').index[0]]

    # Extract ellipse parameters
    try:
        center_x = int(current_frame_data['center_x'])
        center_y = int(current_frame_data['center_y'])
        width = int(current_frame_data['width'])
        height = int(current_frame_data['height'])
        phi = float(current_frame_data['phi'])

        # Draw the ellipse on the frame
        cv2.ellipse(frame, (center_x, center_y), (width, height), phi, 0, 360, (0, 255, 0), 2)
    except ValueError:
        print('could not paint ellipse, missing values')

    # Display the frame
    cv2.imshow("Original Frame", frame)

    # Prompt user to select two points
    print("Please select two points on the frame.")

    # Callback function for mouse events
    def mouse_callback(event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            points.append((x, y))

    # Set up the mouse callback
    cv2.setMouseCallback("Original Frame", mouse_callback)

    # Wait for the user to select two points
    points = []
    while len(points) < 2:
        cv2.waitKey(1)

    # Draw a line between the selected points
    cv2.line(frame, points[0], points[1], (0, 255, 0), 2)
    cv2.imshow("Line Drawn Frame", frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Calculate the rotation angle
    angle = np.arctan2(points[1][1] - points[0][1], points[1][0] - points[0][0]) * 180 / np.pi

    # Create rotation matrix
    rotation_matrix = cv2.getRotationMatrix2D((frame.shape[1] // 2, frame.shape[0] // 2), angle, 1)
    #rotation_matrix[:,2] = 0
    
    # Generate and display video with 50 steps between original and rotated frames
    for step in range(51):
        alpha = step / 50.0
        current_rotation_matrix = cv2.getRotationMatrix2D(
            (frame.shape[1] // 2, frame.shape[0] // 2),
            angle * alpha,
            1
        )
     #   current_rotation_matrix[:,2] = 0
        rotated_frame = cv2.warpAffine(frame, current_rotation_matrix, (frame.shape[1], frame.shape[0]))

        cv2.imshow("Interpolated Rotated Frame", rotated_frame)
        
        cv2.waitKey(100)  # Adjust the wait time to control the playback speed

    # Release resources

    cap.release()
    cv2.destroyAllWindows()

    return rotation_matrix, angle

"""
Example usage
path_to_video = block.le_videos[0]
frame_number = frame_number_to_show
output_path = block.analysis_path / 'frame_rotation.avi'
transformation_matrix, angle = rotate_frame_to_horizontal_with_interpolation(path_to_video, frame_number, block.le_df, output_path=None)
print("Transformation Matrix:")
print(transformation_matrix)
"""


'\nExample usage\npath_to_video = block.le_videos[0]\nframe_number = frame_number_to_show\noutput_path = block.analysis_path / \'frame_rotation.avi\'\ntransformation_matrix, angle = rotate_frame_to_horizontal_with_interpolation(path_to_video, frame_number, block.le_df, output_path=None)\nprint("Transformation Matrix:")\nprint(transformation_matrix)\n'

In [8]:
def apply_rotation_around_center_to_df(eye_df, transformation_matrix, rotation_angle):
    """This is a static method for applying the transformation matrix to eye dataframes within a block class object"""
    original_centers = eye_df[['center_x_corrected', 'center_y_corrected']].values
    original_phi = eye_df['phi'].values
    M = transformation_matrix
    # apply the rotation to xy
    rotated_centers = np.dot(original_centers, M[:, :2].T) + M[:, 2]
    # apply rotation to phi
    rotated_phi = np.rad2deg(original_phi) + rotation_angle
    
    eye_df['center_x_rotated'] = rotated_centers[:,0]
    eye_df['center_y_rotated'] = rotated_centers[:,1]
    eye_df['phi_rotated'] = rotated_phi
    
    return eye_df

In [9]:
# Initialize for left eye video
# get the frames where the pupil is closest to round:
s = block.le_df.width / block.le_df.height
closest_ind = np.argmin(np.abs(s - 1)) # find the index of the value closest to 1
frame_number_to_show = block.le_df.L_eye_frame.iloc[closest_ind]
path_to_video = block.le_videos[0]
frame_number = frame_number_to_show
ellipse_df = block.le_df

left_rotation_matrix, left_angle = rotate_frame_to_horizontal_with_interpolation(path_to_video_file=path_to_video,
                                                                                 frame_number=frame_number,
                                                                                 ellipse_df=ellipse_df,
                                                                                 xflip=True)
print("Left rotation matrix:")
print(left_rotation_matrix)
print("Left angle")
print(left_angle)



Please select two points on the frame.
Left rotation matrix:
[[   0.82171478   -0.56989896  193.82702117]
 [   0.56989896    0.82171478 -139.57921362]]
Left angle
-34.74318015830361


In [10]:
# initialize for right eye video
# get the frames where the pupil is closest to round:
s = block.re_df.width / block.re_df.height
closest_ind = np.argmin(np.abs(s - 1)) # find the index of the value closest to 1
frame_number_to_show = block.re_df.R_eye_frame.iloc[closest_ind]
path_to_video = block.re_videos[0]
frame_number = frame_number_to_show
ellipse_df = block.re_df

right_rotation_matrix, right_angle = rotate_frame_to_horizontal_with_interpolation(path_to_video_file=path_to_video,
                                                                                   frame_number=frame_number,
                                                                                   ellipse_df=ellipse_df,
                                                                                   xflip=True)
print("Right rotation matrix:")
print(right_rotation_matrix)
print("Right angle")
print(right_angle)

Please select two points on the frame.
Right rotation matrix:
[[  0.69586712   0.71817056 -75.03841271]
 [ -0.71817056   0.69586712 302.80646851]]
Right angle
45.90364398751296


In [11]:

block.re_df = apply_rotation_around_center_to_df(eye_df = block.re_df.copy(),
                                           transformation_matrix = right_rotation_matrix,
                                           rotation_angle = right_angle)
block.le_df = apply_rotation_around_center_to_df(eye_df = block.le_df.copy(),
                                           transformation_matrix = left_rotation_matrix,
                                           rotation_angle= left_angle)


In [12]:
block.le_df.phi_rotated

0              NaN
1              NaN
2              NaN
3              NaN
4       -61.259106
           ...    
91948   -13.714678
91949   -13.893080
91950   -13.969731
91951   -13.866422
91952   -13.689242
Name: phi_rotated, Length: 91953, dtype: float64

# Now I should verify this worked...

In [22]:
# verification plot:
original_x_vals = block.re_df['center_x_corrected'].values
new_x_vals = re_df['center_x_rotated'].values
original_y_vals = re_df['center_y_corrected'].values
new_y_vals = re_df['center_y_rotated'].values
bokeh_plotter([original_x_vals, new_x_vals, original_y_vals, new_y_vals],['original_x','new_x', 'original_y_vals', 'new_y_vals'])

In [24]:
original_x_vals = block.le_df['center_x_corrected'].values
new_x_vals = le_df['center_x_rotated'].values
original_y_vals = le_df['center_y_corrected'].values
new_y_vals = le_df['center_y_rotated'].values
bokeh_plotter([original_x_vals, new_x_vals, original_y_vals, new_y_vals],['original_x','new_x', 'original_y_vals', 'new_y_vals'])

In [189]:
# this should rotate + translate the left eye dataframe
df = block.le_df
xy_array = df[['center_x_corrected', 'center_y_corrected']].values
phi_vec = df['phi'].values
M = left_rotation_matrix
rotated_centers = np.dot(xy_array, M[:, :2].T) + M[:, 2]
rotated_phi = np.rad2deg(phi_vec) + left_angle


In [18]:
bokeh_plotter(data_list=[np.diff(np.rad2deg(block.re_df['phi']))],label_list=['phi'])

In [40]:


arr = np.diff(np.rad2deg(block.le_df['phi']))
peaks = np.where(arr > 50)
peaks


(array([19405, 32379, 32411, 32563, 32671, 32706, 32782, 32821, 89832],
       dtype=int64),)

In [14]:
block.re_df['phi_rotated'] = block.re_df['phi_rotated'] + 90  

In [12]:
# right eye inspection before rotation
play_video_with_ellipses(block=block,eye='right', path_to_video=str(path_to_video) ,xflip=True)

In [15]:
# right eye inspection after rotation
path_to_video = [x for x in pathlib.Path(block.re_videos[0]).parent.iterdir() if '.mp4' in str(x.name) and 'DLC' in str(x.name)][0]
play_video_with_ellipses_rotation(block=block,eye='right', path_to_video=str(path_to_video) ,xflip=True, transformation_matrix=right_rotation_matrix)

In [16]:
# left eye inspection
path_to_video = [x for x in pathlib.Path(block.le_videos[0]).parent.iterdir() if '.mp4' in str(x.name) and 'DLC' in str(x.name)][0]
play_video_with_ellipses_rotation(block=block,eye='left', path_to_video=str(path_to_video) ,xflip=True, transformation_matrix=left_rotation_matrix)

# after verification, I want to have a function that exports the re/le df 

In [74]:
def export_eye_dfs(block):
    """
    This function saves the eye dataframes to two csv files
    :param block: The current blocksync class with verifiec re/le dfs
    :return: None
    """
    block.re_df.to_csv(block.analysis_path / 're_df.csv')
    block.le_df.to_csv(block.analysis_path / 'le_df.csv')

export_eye_dfs(block)

In [21]:
for block in block_collection:
    block.get_jitter_reports(export=False, overwrite=False, remove_led_blinks=False, sort_on_loading=True)

jitter report loaded from analysis folder
Jitter report computed - check out re/le_jitter_dict attributes


In [36]:
block.le_jitter_dict.keys()

dict_keys(['top_correlation_values', 'top_correlation_dist', 'y_displacement', 'x_displacement', 'top_correlation_x', 'top_correlation_y'])

In [66]:
# get the R vector for a median absolute deviation calc

jitter_dict = block.re_jitter_dict
# Compute euclidean jitter magnitude
x = np.array(jitter_dict['x_displacement'])
y = np.array(jitter_dict['y_displacement'])
r = np.sqrt(x**2 + y**2)
corr_score = jitter_dict['top_correlation_values']
mean_value = np.mean(r)
mad = np.mean(np.abs(r - mean_value))
extreme_inds = np.where(r > 5 * mad)[0]
filt_r = scipy.signal.medfilt(r, kernel_size=121)
threshold = 3
peaks = np.where(np.abs(np.diff(filt_r)) > threshold)[0]
bokeh_plotter([ r, filt_r, np.abs(np.diff(filt_r))], ['R', 'filtered_R', 'derivative'],
              plot_name='mean absolute deviation', x_axis='frame', y_axis='euclidean displacement', peaks=peaks)
print(len(extreme_inds))

120


In [55]:
threshold = 3
peaks = np.where(np.abs(np.diff(filt_r)) > threshold)[0]
peaks

array([ 3606,  3607,  3608,  3609,  3610,  3734,  4578,  4887, 10638,
       17224, 17228, 20912, 78914, 79687, 88782, 88783, 93412],
      dtype=int64)

In [137]:
# This is the first iteration of the manual frame rotation annotator:

# OLD VERSION OF THE FUNCTION
def rotate_frame_to_horizontal_old(path_to_video_file, frame_number, ellipse_df, xflip=True):
    # Read the video file
    cap = cv2.VideoCapture(path_to_video_file)

    # Check if the video file is opened successfully
    if not cap.isOpened():
        print("Error: Unable to open video file.")
        return None

    # Set the frame position
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)

    # Read the frame
    ret, frame = cap.read()

    # Check if the frame is read successfully
    if not ret:
        print(f"Error: Unable to read frame {frame_number}.")
        cap.release()
        return None
    
    # horizontally flip frame if applicable:
    if xflip:
        frame = cv2.flip(frame, 1)
    # get the original ellipse from the block dataframe
    if 'R_eye_frame' in ellipse_df.columns:
        current_frame_data = ellipse_df.iloc[ellipse_df.query('R_eye_frame == @frame_number').index[0]]
    elif 'L_eye_frame' in ellipse_df.columns:
        current_frame_data = ellipse_df.iloc[ellipse_df.query('L_eye_frame == @frame_number').index[0]]
        
    # Extract ellipse parameters
    try:
        center_x = int(current_frame_data['center_x'])
        center_y = int(current_frame_data['center_y'])
        width = int(current_frame_data['width'])
        height = int(current_frame_data['height'])
        phi = float(current_frame_data['phi'])

        # Draw the ellipse on the frame
        cv2.ellipse(frame, (center_x, center_y), (width, height), phi, 0, 360, (0, 255, 0), 2)
    except ValueError:
        print('could not paint ellipse, missing values')
    
    
    # Display the frame
    cv2.imshow("Original Frame", frame)

    # Prompt user to select two points
    print("Please select two points on the frame.")
    
    # Callback function for mouse events
    def mouse_callback(event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            points.append((x, y))

    # Set up the mouse callback
    cv2.setMouseCallback("Original Frame", mouse_callback)

    # Wait for the user to select two points
    points = []
    while len(points) < 2:
        cv2.waitKey(1)

    # Draw a line between the selected points
    cv2.line(frame, points[0], points[1], (0, 255, 0), 2)
    cv2.imshow("Line Drawn Frame", frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Calculate the rotation angle
    angle = np.arctan2(points[1][1] - points[0][1], points[1][0] - points[0][0]) * 180 / np.pi
        
    # Create rotation matrix
    rotation_matrix = cv2.getRotationMatrix2D((frame.shape[1] // 2, frame.shape[0] // 2), angle, 1)
    
    #rotation_matrix[:, 2] = 0  # Set translation components to zero
    
    # Rotate the frame
    rotated_frame = cv2.warpAffine(frame, rotation_matrix, (frame.shape[1], frame.shape[0]))

    # Display the rotated frame
    cv2.imshow("Rotated Frame", rotated_frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Release the video capture object
    cap.release()

    return rotation_matrix




Please select two points on the frame.
Transformation Matrix:
[[-8.56851544e-01  5.15563217e-01  4.70457322e+02]
 [-5.15563217e-01 -8.56851544e-01  6.10624600e+02]]
